Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (1000, 28, 28) (1000,)
Test set (1000, 28, 28) (1000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 784) (20000, 10)
Validation set (1000, 784) (1000, 10)
Test set (1000, 784) (1000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [20]:
train_subset = 10000
beta = 0.01

graph = tf.Graph()
#using simple gradient discent.
with graph.as_default() :
    tf_train_dataset = tf.constant(train_dataset[:train_subset,:])
    tf_train_labels  = tf.constant(train_labels[:train_subset,:])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset  = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels]))
    biases  = tf.Variable(tf.zeros(num_labels))
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss   = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    regualizer = tf.nn.l2_loss(weights)
    loss   = tf.reduce_mean(loss + beta * regualizer)
    
    optimizer = tf.train.GradientDescentOptimizer(.5).minimize(loss)
    
    train_predictions = tf.nn.softmax(logits)
    valid_predictions = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_predictions  = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
    



In [21]:
num_steps = 801
def accuarcy(predictions, labels):
    return 100.0 * np.sum( np.argmax(predictions,1) == np.argmax(labels,1) ) / labels.shape[0]

with tf.Session(graph=graph) as session :
    
    tf.global_variables_initializer().run()
    print("initialized")
    
    for i in range(num_steps) :
        _, l, predictions = session.run([optimizer, loss, train_predictions])
        
        if i%100 == 0 :
            print ("done %d steps with loss %.4f" % (i, l))
            print ("accuracy on training data = %.2f" % accuarcy(predictions, train_labels[:train_subset,:]))
            print ("validation accuarcy = %.2f" % accuarcy(valid_predictions.eval(), valid_labels))
            print ("test accuarcy = %.2f" % accuarcy(test_predictions.eval(), test_labels))





initialized
done 0 steps with loss 50.5388
accuracy on training data = 8.01
validation accuarcy = 11.00
test accuarcy = 10.30
done 100 steps with loss 12.0614
accuracy on training data = 74.24
validation accuarcy = 72.20
test accuarcy = 80.40
done 200 steps with loss 4.5433
accuracy on training data = 79.27
validation accuarcy = 77.90
test accuarcy = 84.20
done 300 steps with loss 1.9893
accuracy on training data = 82.41
validation accuarcy = 80.00
test accuarcy = 86.20
done 400 steps with loss 1.1255
accuracy on training data = 83.89
validation accuarcy = 82.10
test accuarcy = 87.90
done 500 steps with loss 0.8299
accuracy on training data = 84.38
validation accuarcy = 82.80
test accuarcy = 87.70
done 600 steps with loss 0.7274
accuracy on training data = 84.62
validation accuarcy = 83.30
test accuarcy = 87.70
done 700 steps with loss 0.6915
accuracy on training data = 84.67
validation accuarcy = 83.20
test accuarcy = 88.00
done 800 steps with loss 0.6788
accuracy on training data = 8

In [40]:
train_subset = 10000
beta = 0.01
batch_size = 128
num_nodes = 1024
# using sophiastic gradient discent and l2 regularization and hidden layer, and dropout.

graph = tf.Graph()


with graph.as_default() :
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels  = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset  = tf.constant(test_dataset)
    
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size, num_nodes]))
    weights2 = tf.Variable(tf.truncated_normal([ num_nodes, num_labels ]))
    biases1  = tf.Variable(tf.zeros(num_nodes))
    biases2  = tf.Variable(tf.zeros(num_labels))
    
    logits1    = tf.matmul(tf_train_dataset, weights1) + biases1
    relu_layer = tf.nn.relu(logits1)
    keep_prob  = tf.placeholder(tf.float32)
    relu_layer_dropout = tf.nn.dropout(relu_layer, keep_prob)
    logits2    = tf.matmul(relu_layer_dropout, weights2) + biases2 
    
    loss   = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits2))
    
    regualizer = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)
    loss   = tf.reduce_mean(loss + beta * regualizer)
    
    optimizer = tf.train.GradientDescentOptimizer(.5).minimize(loss)
    
    train_predictions = tf.nn.softmax(logits2)
    
    # valid predictions
    logits1    = tf.matmul(tf_valid_dataset, weights1) + biases1
    relu_layer = tf.nn.relu(logits1)
    logits2    = tf.matmul(relu_layer, weights2) + biases2 
    valid_predictions = tf.nn.softmax(logits2)
    # test predictions
    logits1    = tf.matmul(tf_test_dataset, weights1) + biases1
    relu_layer = tf.nn.relu(logits1)
    logits2    = tf.matmul(relu_layer, weights2) + biases2 
    test_predictions  = tf.nn.softmax(logits2)

In [41]:
num_steps = 801


with tf.Session(graph=graph) as session :
    
    tf.global_variables_initializer().run()
    print("initialized")
    
    for i in range(num_steps) :
        
        offset = (i * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_data   = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = { tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : .5 }
        
        _, l, predictions = session.run([optimizer, loss, train_predictions], feed_dict=feed_dict)
        
        if i%100 == 0 :
            print ("done %d steps with loss %.4f" % (i, l))
            print ("accuracy on training data = %.2f" % accuarcy(predictions,
                                                                 train_labels[offset:(offset + batch_size),:]))
            print ("validation accuarcy = %.2f" % accuarcy(valid_predictions.eval(), valid_labels))
            print ("test accuarcy = %.2f" % accuarcy(test_predictions.eval(), test_labels))





initialized
done 0 steps with loss 3628.9812
accuracy on training data = 7.81
validation accuarcy = 29.50
test accuarcy = 30.50
done 100 steps with loss 1170.1752
accuracy on training data = 72.66
validation accuarcy = 80.00
test accuarcy = 85.10
done 200 steps with loss 427.9018
accuracy on training data = 75.00
validation accuarcy = 82.10
test accuarcy = 86.20
done 300 steps with loss 155.1611
accuracy on training data = 81.25
validation accuarcy = 82.10
test accuarcy = 88.00
done 400 steps with loss 57.3587
accuracy on training data = 81.25
validation accuarcy = 83.80
test accuarcy = 90.60
done 500 steps with loss 21.4804
accuracy on training data = 85.94
validation accuarcy = 85.00
test accuarcy = 91.00
done 600 steps with loss 8.4197
accuracy on training data = 82.03
validation accuarcy = 84.50
test accuarcy = 89.70
done 700 steps with loss 3.6003
accuracy on training data = 86.72
validation accuarcy = 84.60
test accuarcy = 90.20
done 800 steps with loss 1.8610
accuracy on trainin

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
